# Compute degrees of belief with Multi-Layer Belief Model in Python

Based on Daira Pinto Prieto, Ronald de Haan, and Aybüke Özgün, **A Belief Model for Conflicting and Uncertain Evidence: Connecting Dempster-Shafer Theory and the Topology of Evidence**. In: Proceedings of the 20th International Conference on Principles of Knowledge Representation and Reasoning (KR 2023), 2023 *(To appear)*. Pre-print version available: https://arxiv.org/abs/2306.03532

This notebook presents an implementation of the Multi-Layer Belief model from Pinto Prieto et al. (2023) with an extended example. It provides a Topological class for necessary computations and a Model class containing intermediate functions. Moreover, the notebook provides a concrete example illustrating the generation of six distinct models of justified beliefs from a shared set of evidence, depending on the different parameter options of the Multi-Layer model. At the end of the notebook, the reader can compare belief degrees assigned by each model to certain propositions. While the results are accurate, please note that this Jupyter notebook is a work in progress.

<sub> I acknowledge Arthur Boixel for his assistance with the coding structure and implementation of this notebook. </sub>

### Importing necessary packages and definition of auxility functions.

In [1]:
import sys
import ast
from functools import reduce
from itertools import chain, combinations 

This two auxiliary functions will help us to work with sets and dictionaries. Each set can be uniquely identified by a string, and thus used as a dictionary key.

In [3]:
def get_key(s):
    """
    This method gives a unique tuple identifier to a set.

    Args:
        s (set): Set.
        
    Returns:
        k (string): Unique string identifier for s.
    """
    return str(s)

def get_set(k):
    """
    This method retrieves a set from a unique string identifier.

    Args:
        k (string): Unique string identifier.
        
    Returns:
        s (set): Corresponding set.
    """
    elements = ast.literal_eval(k)
    return set(elements)

# Example usage
s = {'b', 'a'}
k = get_key(s)
#print(type(k))
#print(k)
s1 = get_set(k)
#print(s1)
#print(s == s1)

### Topology class and related functions


In [4]:
class Topology:
    
    """ This class defines an object Topology """
    
   
    def __init__(self, E=None, S=None):
        """
        This is the constructor for Topology.

        Args:
            E (list/dictionary): Lis of pieces of evidence / Dictionary with pieces of evidence as keys and certainty degree as values.
            S (set, optional): Space. Defaults to None.

        Returns:
            None
        """

        if E is None:
            raise ValueError("E (Evidence) argument must be provided.")

        if isinstance(E, dict):
            E = [get_set(k) for k in E.keys()]

        if S is None:
            S = set().union(*E)

        self.S = S
        self.E = E
    
    
    def get_subbasis(self):
        """
        This methods returns the subbasis of the topology generated by a collection of pieces of evidence. This is exactly the list of pieces of evidence. 

        Returns:
            E (list):   Evidence
        """

        #if type(self.E) == dict:
        #    return [get_set(k) for k in self.E.keys()]
        
        return self.E
    
    def get_basis(self):
        """
        This method computes the basis of the topology generated by the evidence. Namely, the closure by finite intersections of the subbais.
        
        Returns:
            set (set) : The basis of the topology.
        """
        
        basis = self.get_subbasis() # list of sets
        updated = True

        while updated:
            updated = False  # Set to False initially

        for (s1, s2) in combinations(basis, 2):
            inter = s1.intersection(s2)
            if inter not in basis:
                basis.append(inter)
                updated = True

        return basis

    def get_topology(self):
        """
        This methods returns the topology generated by the evidence. Namely, the closure by finite arbitrary intersections of the set basis \cup \emptyset \ S.

        Returns:
            set (set) : The topology.
        """

        basis = self.get_basis()
        topology = basis  # Initialize the topology with the basis elements
        if set() not in topology:
            topology.append(set())  # Add the empty set
        if self.S not in topology:
            topology.append(self.S) 
        
        updated = True

        while updated:
            updated = False

            for size in range(2, len(basis) + 1):
                # Generate all combinations of size `size` from the basis elements
                combinations_size = combinations(basis, size)

                for combination in combinations_size:
                    union = set().union(*combination)  # Compute the union of the combination elements
                    if union not in topology:
                        topology.append(union)  # Add the union to the topology
                        updated = True

        return topology

    def get_dense_sets(self):
        """
        This method computes the set of dense sets of the topology. 
        A set S is dense if (i) it belongs to the topology 
        and (ii) has a non-empty intersection with every set in the topology.

        Returns:
            set (set) : The set of dense sets of the topology.
        """
        topology = self.get_topology()
        topology.remove(set())
        dense_sets = []

        for candidate_set in topology:
            is_dense = True

            for other_set in topology:
                if not candidate_set.intersection(other_set):
                    is_dense = False
                    break

            if is_dense:
                dense_sets.append(candidate_set)

        return dense_sets        


    def print(self):
        """
        This methods prints the topology.

        Returns:
            None
        """

        print("Subbasis: " + str(self.get_subbasis()))
        print("Basis: " + str(self.get_basis()))
        print("Topology: ", str(self.get_topology()))
        print("Dense sets: ", str(self.get_dense_sets()))

def minimal_dense_set(sets):
    """
    This function computes the smallest dense set from a list of sets.
    A set S is dense if (i) it belongs to the topology and (ii) has a non-empty intersection with every set in the topology.

    Args:
        sets (list): A list of sets.

    Returns:
        set: The smallest dense set of the topology.
    """
    topology = Topology(sets)

    dense_sets = topology.get_dense_sets()

    return min(dense_sets, key=len, default=topology.S)

Trying out the Topology class.

In [5]:
S = {'A', 'B', 'C'}                                           # Space set of states
E1 = {get_key({'A', 'B'}): 0.1, get_key({'B', 'C'}): 0.2, get_key({'C'}): 0.3}      # Dict of Quantitative Evidence
E2 = [{'B', 'A'}, {'C', 'B'}, {'C'}]                                                # List of evidence

topology1 = Topology(E1, S)
# Print object attributes
print("Example with E1 = {get_key({'A', 'B'}): 0.1, get_key({'B', 'C'}): 0.2, get_key({'C'}): 0.3}: ")
topology1.print()

""" topology2 = Topology(E2, S)
# Print object attributes
print("Example with E2 = [{'B', 'A'}, {'C', 'B'}, {'C'}]: ")
topology2.print() """

# Print minimal_dense_set function
minimal_dense_set_test = minimal_dense_set([{'B', 'A'}, {'C', 'B'}, {'C'}])
print("Minimal dense set of [{'B', 'A'}, {'C', 'B'}, {'C'}] = " + str(minimal_dense_set_test))


Example with E1 = {get_key({'A', 'B'}): 0.1, get_key({'B', 'C'}): 0.2, get_key({'C'}): 0.3}: 
Subbasis: [{'B', 'A'}, {'B', 'C'}, {'C'}]
Basis: [{'B', 'A'}, {'B', 'C'}, {'C'}, {'B'}, set()]
Topology:  [{'B', 'A'}, {'B', 'C'}, {'C'}, {'B'}, set(), {'A', 'B', 'C'}]
Dense sets:  [{'B', 'C'}, {'A', 'B', 'C'}]
Minimal dense set of [{'B', 'A'}, {'C', 'B'}, {'C'}] = {'B', 'C'}


### Multi-Layer Belief Model class

The `Model` class contains all relevant information and methods to generate examples of the Multi-Layer Belief Model.

In [6]:
class Model:
    
    """ This class defines an object Model containing all the information
        required to compute some degree of belief from a collection of uncertain pieces of evidence. 
        The user must provide as input the Justification frame (the criteria to consider an argument 
        good enough to justify degree of belief on those propositions implied by this argument); and 
        allocation fucntion (how we will interpret the sets of pieces of evience). For example, to 
        obtain the equivalent results to Demspter's rule of combination, the user must choose 
        'dempster_shafer' as Justification frame and 'set.intersection' as allocation function. """
    
   
    def __init__(self, S, E, J, f):
        """
        This is the constructor for Model.

        Args:
            S (set):          Space.
            E (dict):         Quantitative evidence.
            J (string):       Justification frame (dempster_shafer/strong_denseness = dense and non-dense sets / dense sets of the topology). 
            f (function):     Allocation function (set.intersection/set.union/minimal_dense_set)

        Returns:
            None
        """
        
        self.space = S
        self.quantitative_evidence = E
        self.justification_frame = J
        self.allocation_function = f
        self.delta = None
        self.delta_tau = None
        self.delta_J = None
        

    
    
    def get_delta(self):
        """
        This method computes the delta function. For each combination of pieces of evidence,
        a corresponding delta value is computed. 
        
        Returns:
            dict (str, double) : For each combination of evidence, the delta value
        """
        
        # Avoid redundant computations
        if self.delta != None:
            return self.delta
        
        n = len(self.quantitative_evidence)
        pieces_of_evidence = [get_set(e) for e in self.quantitative_evidence.keys()]
        self.delta = {}
        
        for r in range(0,n+1):                               # For every size r of combination of pieces of evidence
            for combination in combinations(pieces_of_evidence, r):  # For all combinations of size r

                remaining_evidence = [e for e in pieces_of_evidence if e not in list(combination)]

                value = 1.0
                
                for evidence in combination:  # For each piece of evidence in the combination
                    value *= self.quantitative_evidence[get_key(evidence)] # The value is multiplied by ...
                
                for evidence in remaining_evidence: # For each piece of evidence not in the combination
                    value *= 1 - self.quantitative_evidence[get_key(evidence)] # The value is multiplied by ...

                self.delta[str(list(combination))] = value   # We add the delta for this combination
        
        return self.delta
    
    def get_delta_tau(self):
        """
        This method computes the delta tau function. The allocation function is applied to each 
        argument of the delta function. The results constitutes the entries of the delta tau function.
        For each entry e, delta_tau(e) = sum(delta(a) if allocation(a) = e)
        
        Returns:
            dict(str, double) : For each combination of evidence, the delta value argument/ element of topology
        """

        # Avoid redundant computations
        if self.delta_tau != None:
            return self.delta_tau
        
        delta = self.get_delta().items()
        self.delta_tau = {}
        
        # Creation of new keys
        new_keys = []        
        for (combination,value) in delta: # Browsig entries of delta function
            argument = ast.literal_eval(combination) # Get the delta entry as a set

            if not argument:  # If delta entry is the empty set, new entry is S
                entry = self.space
            elif self.allocation_function == minimal_dense_set:
                entry = self.allocation_function(argument) # Compute allocation function on list
            else:
                entry = reduce(self.allocation_function, argument)  # Compute allocation function on list
            
            key = get_key(entry)


            if self.delta_tau.get(key):
                self.delta_tau[key] += value
            else:
                self.delta_tau[key] = value
            
            
            
        return self.delta_tau
    
    def get_justification_frame(self):
        """
        This method computes the justifications frame of the model.  
        
        Returns:
            list of sets
        """

        topology = Topology(self.quantitative_evidence, self.space)
        if self.justification_frame == 'dempster_shafer':
            justification_set = topology.get_topology()
            justification_set.remove(set())  # Remove the empty set from the topology
        elif self.justification_frame == 'strong_denseness':
            justification_set = topology.get_dense_sets()
        else:
            raise ValueError("Invalid value for J (Justification Frame). Must be 'dempster_shafer' or 'strong_denseness'.")
        
        return justification_set
    
    def get_normalization_factor(self):
        """
        This method computes the total sum of the frame of justifications.  
        
        Returns:
            denominator (double)
        """
        delta_tau = self.get_delta_tau()
        justification_set = self.get_justification_frame()
        
        denominator = sum(delta_tau[get_key(j)] for j in justification_set if get_key(j) in delta_tau) # Since delta_tau only contains elements with non-zero value, there may be j that are not in delta_tau
        
        return denominator

    
    def get_delta_J(self):
        """
        This method computes the deltaJ functions. 
        For each justification j in J, delta_J(j) = delta_tau(j) / sum(delta_tau(k) for k in J)
        
        Returns:
            dict (str, double) : For each combination of evidence, the delta value argument/ element of topology
        """

        # Avoid redundant computations
        if self.delta_J != None:
            return self.delta_J
        
    
        self.delta_J = {}
        delta_tau = self.get_delta_tau()
        """ topology = Topology(self.quantitative_evidence, self.space)
        if self.justification_frame == 'dempster_shafer':
            justification_set = topology.get_topology()
            justification_set.remove(set())  # Remove the empty set from the topology
        elif self.justification_frame == 'strong_denseness':
            justification_set = topology.get_dense_sets()
        else:
            raise ValueError("Invalid value for J (Justification Frame). Must be 'dempster_shafer' or 'strong_denseness'.")      
        
        
        denominator = sum(delta_tau[get_key(j)] for j in justification_set if get_key(j) in delta_tau) # Since delta_tau only contains elements with non-zero value, there may be j that are not in delta_tau
        print("Denominator: " + str(denominator)) """

        justification_set = self.get_justification_frame()
        denominator = self.get_normalization_factor()

        
        for justification in justification_set: # For all justification j
            key = get_key(justification)
            if key in delta_tau:                                    # If delta_tau(j) exists (has non-zero value)
                self.delta_J[key] = delta_tau[key] / denominator   #    Compute delta_j(j)
                
        testSum = sum(value for value in self.delta_J.values())
        print("Sum values: " + str(testSum))
        
        
        return self.delta_J
    

    def clear(self):
        """
        This method reinitializes the different tables of the objects.
        """

        self.delta = None
        self.delta_tau = None
        self.delta_J = None
    
    
    def degree_of_belief(self, proposition):
        """
        This method computes the degree of belief of a given proposition. 
        For a proposition p, degree_of_belief(p) = sum(delta_J(j) for j subset p)
        
        Returns:
            float : The degree of belief of p
        """
        
        delta_J = self.get_delta_J()
        
        return sum(delta_J[j] for j in delta_J if get_set(j).issubset(proposition))
        
    

### Example

Let's start by importing a useful package to visualize our results.

In [7]:
from tabulate import tabulate

Next we define our set of possible states and our set of quantitative evidence.

In [12]:
S = {'dp', 'sp', 'dm', 'sm', 'do'}                                           # Space set of states
E = {get_key({'dp', 'dm', 'do'}): 0.9, get_key({'dm', 'sm'}): 0.75, get_key({'dp', 'sp'}): 0.45}      # Quantitative Evidence element subset of S

The class contains a method allowing you to reinitialize the different tables.

In [14]:
f = set.intersection
J = 'dempster_shafer'
model = Model(S,E,J,f)
model.clear()

Now let's create the six possible models for this input. In this example, Delta function returns the same values for all the models, since it only depends on S and E. 

#### Model 1: Dempster-Shafer justification frame + intersection as allocation function

In [22]:
J_1 = 'dempster_shafer'               # Justification Frame element subset of s
f_1 = set.intersection                    # Allocation function

model_1 = Model(S, E, J_1, f_1)

# Print object attributes
print("Space: ", model_1.space)
print("Quantitative Evidence: ", model_1.quantitative_evidence)
print("Justification Frame: ", model_1.justification_frame)
print("Allocation Function: ", model_1.allocation_function)

# Print delta function
table = [[key, value] for key, value in model_1.get_delta().items()]

print("\nDelta:\n")
print(tabulate(table, headers=['Set', 'Value'], tablefmt='orgtbl'))

# Print delta_tau function
table = [[key, value] for key, value in model_1.get_delta_tau().items()]

print("\nDelta tau:\n")
print(tabulate(table, headers=['Argument', 'Value'], tablefmt='orgtbl'))

# Print delta_J function
delta_J1 = model_1.get_delta_J()
table = [[key, value] for key, value in delta_J1.items()]

print("\nDelta J:\n")
print(tabulate(table, headers=['Justification', 'Value'], tablefmt='orgtbl'))

Space:  {'dp', 'do', 'sp', 'dm', 'sm'}
Quantitative Evidence:  {"{'dp', 'do', 'dm'}": 0.9, "{'sm', 'dm'}": 0.75, "{'dp', 'sp'}": 0.45}
Justification Frame:  dempster_shafer
Allocation Function:  <method 'intersection' of 'set' objects>

Delta:

| Set                                              |   Value |
|--------------------------------------------------+---------|
| []                                               | 0.01375 |
| [{'dp', 'do', 'dm'}]                             | 0.12375 |
| [{'sm', 'dm'}]                                   | 0.04125 |
| [{'dp', 'sp'}]                                   | 0.01125 |
| [{'dp', 'do', 'dm'}, {'sm', 'dm'}]               | 0.37125 |
| [{'dp', 'do', 'dm'}, {'dp', 'sp'}]               | 0.10125 |
| [{'sm', 'dm'}, {'dp', 'sp'}]                     | 0.03375 |
| [{'dp', 'do', 'dm'}, {'sm', 'dm'}, {'dp', 'sp'}] | 0.30375 |

Delta tau:

| Argument                       |   Value |
|--------------------------------+---------|
| {'dp', 'do', 'sp', '

#### Model 2: Dempster-Shafer justification frame + union as allocation function

In [23]:
J_2 = 'dempster_shafer'               # Justification Frame element subset of s
f_2 = set.union                    # Allocation function

model_2 = Model(S, E, J_2, f_2)

# Print object attributes
print("Space: ", model_2.space)
print("Quantitative Evidence: ", model_2.quantitative_evidence)
print("Justification Frame: ", model_2.justification_frame)
print("Allocation Function: ", model_2.allocation_function)

# Print delta function
table = [[key, value] for key, value in model_2.get_delta().items()]

print("\nDelta:\n")
print(tabulate(table, headers=['Set', 'Value'], tablefmt='orgtbl'))

# Print delta_tau function
table = [[key, value] for key, value in model_2.get_delta_tau().items()]

print("\nDelta tau:\n")
print(tabulate(table, headers=['Argument', 'Value'], tablefmt='orgtbl'))

# Print delta_J function
delta_J2 = model_2.get_delta_J()
table = [[key, value] for key, value in delta_J2.items()]
print("\nDelta J:\n")
print(tabulate(table, headers=['Justification', 'Value'], tablefmt='orgtbl'))

Space:  {'dp', 'do', 'sp', 'dm', 'sm'}
Quantitative Evidence:  {"{'dp', 'do', 'dm'}": 0.9, "{'sm', 'dm'}": 0.75, "{'dp', 'sp'}": 0.45}
Justification Frame:  dempster_shafer
Allocation Function:  <method 'union' of 'set' objects>

Delta:

| Set                                              |   Value |
|--------------------------------------------------+---------|
| []                                               | 0.01375 |
| [{'dp', 'do', 'dm'}]                             | 0.12375 |
| [{'sm', 'dm'}]                                   | 0.04125 |
| [{'dp', 'sp'}]                                   | 0.01125 |
| [{'dp', 'do', 'dm'}, {'sm', 'dm'}]               | 0.37125 |
| [{'dp', 'do', 'dm'}, {'dp', 'sp'}]               | 0.10125 |
| [{'sm', 'dm'}, {'dp', 'sp'}]                     | 0.03375 |
| [{'dp', 'do', 'dm'}, {'sm', 'dm'}, {'dp', 'sp'}] | 0.30375 |

Delta tau:

| Argument                       |   Value |
|--------------------------------+---------|
| {'dp', 'do', 'sp', 'dm', 's

#### Model 3: Dempster-Shafer justification frame + minimal dense set as allocation function

In [25]:
J_3 = 'dempster_shafer'               # Justification Frame element subset of s
f_3 = minimal_dense_set                    # Allocation function

model_3 = Model(S, E, J_3, f_3)

# Print object attributes
print("Space: ", model_3.space)
print("Quantitative Evidence: ", model_3.quantitative_evidence)
print("Justification Frame: ", model_3.justification_frame)
print("Allocation Function: ", model_3.allocation_function)

# Print delta function
table = [[key, value] for key, value in model_3.get_delta().items()]

print("\nDelta:\n")
print(tabulate(table, headers=['Set', 'Value'], tablefmt='orgtbl'))

# Print delta_tau function
table = [[key, value] for key, value in model_3.get_delta_tau().items()]

print("\nDelta tau:\n")
print(tabulate(table, headers=['Argument', 'Value'], tablefmt='orgtbl'))

# Print delta_J function
delta_J3 = model_3.get_delta_J()
table = [[key, value] for key, value in delta_J3.items()]

print("\nDelta J:\n")
print(tabulate(table, headers=['Justification', 'Value'], tablefmt='orgtbl'))

Space:  {'dp', 'do', 'sp', 'dm', 'sm'}
Quantitative Evidence:  {"{'dp', 'do', 'dm'}": 0.9, "{'sm', 'dm'}": 0.75, "{'dp', 'sp'}": 0.45}
Justification Frame:  dempster_shafer
Allocation Function:  <function minimal_dense_set at 0x000001A4461084C0>

Delta:

| Set                                              |   Value |
|--------------------------------------------------+---------|
| []                                               | 0.01375 |
| [{'dp', 'do', 'dm'}]                             | 0.12375 |
| [{'sm', 'dm'}]                                   | 0.04125 |
| [{'dp', 'sp'}]                                   | 0.01125 |
| [{'dp', 'do', 'dm'}, {'sm', 'dm'}]               | 0.37125 |
| [{'dp', 'do', 'dm'}, {'dp', 'sp'}]               | 0.10125 |
| [{'sm', 'dm'}, {'dp', 'sp'}]                     | 0.03375 |
| [{'dp', 'do', 'dm'}, {'sm', 'dm'}, {'dp', 'sp'}] | 0.30375 |

Delta tau:

| Argument                       |   Value |
|--------------------------------+---------|
| {'dp', 'do

#### Model 4: Strong denseness justification frame + intersection as allocation function

In [27]:
J_4 = 'strong_denseness'               # Justification Frame element subset of s
f_4 = set.intersection                    # Allocation function

model_4 = Model(S, E, J_4, f_4)

# Print object attributes
print("Space: ", model_4.space)
print("Quantitative Evidence: ", model_4.quantitative_evidence)
print("Justification Frame: ", model_4.justification_frame)
print("Allocation Function: ", model_4.allocation_function)

# Print delta function
table = [[key, value] for key, value in model_4.get_delta().items()]

print("\nDelta:\n")
print(tabulate(table, headers=['Set', 'Value'], tablefmt='orgtbl'))

# Print delta_tau function
table = [[key, value] for key, value in model_4.get_delta_tau().items()]

print("\nDelta tau:\n")
print(tabulate(table, headers=['Argument', 'Value'], tablefmt='orgtbl'))

# Print delta_J function
delta_J4 = model_4.get_delta_J()
table = [[key, value] for key, value in delta_J4.items()]

print("\nDelta J:\n")
print(tabulate(table, headers=['Justification', 'Value'], tablefmt='orgtbl'))

Space:  {'dp', 'do', 'sp', 'dm', 'sm'}
Quantitative Evidence:  {"{'dp', 'do', 'dm'}": 0.9, "{'sm', 'dm'}": 0.75, "{'dp', 'sp'}": 0.45}
Justification Frame:  strong_denseness
Allocation Function:  <method 'intersection' of 'set' objects>

Delta:

| Set                                              |   Value |
|--------------------------------------------------+---------|
| []                                               | 0.01375 |
| [{'dp', 'do', 'dm'}]                             | 0.12375 |
| [{'sm', 'dm'}]                                   | 0.04125 |
| [{'dp', 'sp'}]                                   | 0.01125 |
| [{'dp', 'do', 'dm'}, {'sm', 'dm'}]               | 0.37125 |
| [{'dp', 'do', 'dm'}, {'dp', 'sp'}]               | 0.10125 |
| [{'sm', 'dm'}, {'dp', 'sp'}]                     | 0.03375 |
| [{'dp', 'do', 'dm'}, {'sm', 'dm'}, {'dp', 'sp'}] | 0.30375 |

Delta tau:

| Argument                       |   Value |
|--------------------------------+---------|
| {'dp', 'do', 'sp', 

#### Model 5: Strong denseness justification frame + union as allocation function

In [28]:
J_5 = 'strong_denseness'               # Justification Frame element subset of s
f_5 = set.union                    # Allocation function

model_5 = Model(S, E, J_5, f_5)

# Print object attributes
print("Space: ", model_5.space)
print("Quantitative Evidence: ", model_5.quantitative_evidence)
print("Justification Frame: ", model_5.justification_frame)
print("Allocation Function: ", model_5.allocation_function)

# Print delta function
table = [[key, value] for key, value in model_5.get_delta().items()]

print("\nDelta:\n")
print(tabulate(table, headers=['Set', 'Value'], tablefmt='orgtbl'))

# Print delta_tau function
table = [[key, value] for key, value in model_5.get_delta_tau().items()]

print("\nDelta tau:\n")
print(tabulate(table, headers=['Argument', 'Value'], tablefmt='orgtbl'))

# Print delta_J function
delta_J5 = model_5.get_delta_J()
table = [[key, value] for key, value in delta_J5.items()]

print("\nDelta J:\n")
print(tabulate(table, headers=['Justification', 'Value'], tablefmt='orgtbl'))

Space:  {'dp', 'do', 'sp', 'dm', 'sm'}
Quantitative Evidence:  {"{'dp', 'do', 'dm'}": 0.9, "{'sm', 'dm'}": 0.75, "{'dp', 'sp'}": 0.45}
Justification Frame:  strong_denseness
Allocation Function:  <method 'union' of 'set' objects>

Delta:

| Set                                              |   Value |
|--------------------------------------------------+---------|
| []                                               | 0.01375 |
| [{'dp', 'do', 'dm'}]                             | 0.12375 |
| [{'sm', 'dm'}]                                   | 0.04125 |
| [{'dp', 'sp'}]                                   | 0.01125 |
| [{'dp', 'do', 'dm'}, {'sm', 'dm'}]               | 0.37125 |
| [{'dp', 'do', 'dm'}, {'dp', 'sp'}]               | 0.10125 |
| [{'sm', 'dm'}, {'dp', 'sp'}]                     | 0.03375 |
| [{'dp', 'do', 'dm'}, {'sm', 'dm'}, {'dp', 'sp'}] | 0.30375 |

Delta tau:

| Argument                       |   Value |
|--------------------------------+---------|
| {'dp', 'do', 'sp', 'dm', '

#### Model 6: Strong denseness justification frame + minimal dense set as allocation function

In [29]:
J_6 = 'strong_denseness'               # Justification Frame element subset of s
f_6 = minimal_dense_set                    # Allocation function

model_6 = Model(S, E, J_6, f_6)

# Print object attributes
print("Space: ", model_6.space)
print("Quantitative Evidence: ", model_6.quantitative_evidence)
print("Justification Frame: ", model_6.justification_frame)
print("Allocation Function: ", model_6.allocation_function)

# Print delta function
table = [[key, value] for key, value in model_6.get_delta().items()]

print("\nDelta:\n")
print(tabulate(table, headers=['Set', 'Value'], tablefmt='orgtbl'))

# Print delta_tau function
table = [[key, value] for key, value in model_6.get_delta_tau().items()]

print("\nDelta tau:\n")
print(tabulate(table, headers=['Argument', 'Value'], tablefmt='orgtbl'))

# Print delta_J function
delta_J6 = model_6.get_delta_J()
table = [[key, value] for key, value in delta_J6.items()]

print("\nDelta J:\n")
print(tabulate(table, headers=['Justification', 'Value'], tablefmt='orgtbl'))

Space:  {'dp', 'do', 'sp', 'dm', 'sm'}
Quantitative Evidence:  {"{'dp', 'do', 'dm'}": 0.9, "{'sm', 'dm'}": 0.75, "{'dp', 'sp'}": 0.45}
Justification Frame:  strong_denseness
Allocation Function:  <function minimal_dense_set at 0x000001A4461084C0>

Delta:

| Set                                              |   Value |
|--------------------------------------------------+---------|
| []                                               | 0.01375 |
| [{'dp', 'do', 'dm'}]                             | 0.12375 |
| [{'sm', 'dm'}]                                   | 0.04125 |
| [{'dp', 'sp'}]                                   | 0.01125 |
| [{'dp', 'do', 'dm'}, {'sm', 'dm'}]               | 0.37125 |
| [{'dp', 'do', 'dm'}, {'dp', 'sp'}]               | 0.10125 |
| [{'sm', 'dm'}, {'dp', 'sp'}]                     | 0.03375 |
| [{'dp', 'do', 'dm'}, {'sm', 'dm'}, {'dp', 'sp'}] | 0.30375 |

Delta tau:

| Argument                       |   Value |
|--------------------------------+---------|
| {'dp', 'd

#### Degree of Belief according to Models 1 - 6

In [21]:
# Define propositions
p1 = {'do', 'dm', 'dp'}
p2 = {'sp', 'dp'}
p3 = {'dm', 'sm'}
p4 = {'dm', 'dp'}

# Define the table degree of belief
table_belief = [
    ["(1)", "{dp,do,dm}", model_1.degree_of_belief(p1), model_2.degree_of_belief(p1), 
     model_3.degree_of_belief(p1), model_4.degree_of_belief(p1), model_5.degree_of_belief(p1), model_6.degree_of_belief(p1)],
    ["(2)", "{sp,dp}", model_1.degree_of_belief(p2), model_2.degree_of_belief(p2),
     model_3.degree_of_belief(p2), model_4.degree_of_belief(p2), model_5.degree_of_belief(p2), model_6.degree_of_belief(p2)],
    ["(3)", "{dm,sm}", model_1.degree_of_belief(p3), model_2.degree_of_belief(p3),
     model_3.degree_of_belief(p3), model_4.degree_of_belief(p3), model_5.degree_of_belief(p3), model_6.degree_of_belief(p3)],
    ["(4)", "{dp,dm}", model_1.degree_of_belief(p4), model_2.degree_of_belief(p4), 
     model_3.degree_of_belief(p4), model_4.degree_of_belief(p4), model_5.degree_of_belief(p4), model_6.degree_of_belief(p4)]
]

# Define the table belief headers
headers_belief = ["Proposition", "P", "Model 1", "Model 2", "Model 3", "Model 4", "Model 5", "Model 6"]  # Empty headers for the horizontal lines
horizontal_line = ["-"] * 8  # Horizontal line

# Compute uncertainty of the models
uncertainty_1 = delta_J1[get_key(S)]
uncertainty_2 = delta_J2[get_key(S)]
uncertainty_3 = delta_J3[get_key(S)]
uncertainty_4 = delta_J4[get_key(S)]
uncertainty_5 = delta_J5[get_key(S)]
uncertainty_6 = delta_J6[get_key(S)]

# Compute Normalization factor of the models

denominator_1 = model_1.get_normalization_factor()
denominator_2 = model_2.get_normalization_factor()
denominator_3 = model_3.get_normalization_factor()
denominator_4 = model_4.get_normalization_factor()
denominator_5 = model_5.get_normalization_factor()
denominator_6 = model_6.get_normalization_factor()

# Define the table of other metrics
table_metrics = [
    ["Uncertainty", "S", uncertainty_1, uncertainty_2,
      uncertainty_3, uncertainty_4, uncertainty_5, uncertainty_6],
    ["Normalization factor", "J", denominator_1, denominator_2,
     denominator_3, denominator_4, denominator_5, denominator_6]
]

headers_metrics = ["", "", "Model 1", "Model 2", "Model 3", "Model 4", "Model 5", "Model 6"]  # Empty headers for the horizontal lines
horizontal_line = ["-"] * 8  # Horizontal line

# Print the table
print("\nDegree of Belief:\n")
print(tabulate(table_belief, headers=headers_belief, tablefmt='orgtbl'))
print("\n(1): Dynamic object")
print("(2): Pedestrian")
print("(3): Motorbike")
print("(4): Dynamic person-motorbike\n")
print("\nOther metrics:\n")
print(tabulate(table_metrics, headers=headers_metrics, tablefmt='orgtbl'))


Degree of Belief:

| Proposition   | P          |   Model 1 |   Model 2 |   Model 3 |   Model 4 |   Model 5 |   Model 6 |
|---------------+------------+-----------+-----------+-----------+-----------+-----------+-----------|
| (1)           | {dp,do,dm} |  0.9      |   0.12375 |   0.9     |       0.9 |  0.130607 |  0.9      |
| (2)           | {sp,dp}    |  0.169811 |   0.01125 |   0.1125  |       0   |  0        |  0        |
| (3)           | {dm,sm}    |  0.622642 |   0.04125 |   0.4125  |       0   |  0        |  0        |
| (4)           | {dp,dm}    |  0.713208 |   0       |   0.77625 |       0   |  0        |  0.639474 |

(1): Dynamic object
(2): Pedestrian
(3): Motorbike
(4): Dynamic person-motorbike


Other metrics:

|                      |    |   Model 1 |   Model 2 |   Model 3 |   Model 4 |   Model 5 |   Model 6 |
|----------------------+----+-----------+-----------+-----------+-----------+-----------+-----------|
| Uncertainty          | S  | 0.0207547 |    0.3175 |   0.